# Meme Humour Classification: Deep-learning-based Multi-modal Fusion with Humor Semantics

## 1. import

In [3]:
import os
import sys

# TODO: change this to the path to your homework folder
GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive')
print(os.listdir(GOOGLE_DRIVE_PATH))

os.chdir(GOOGLE_DRIVE_PATH)

['4', 'Colab Notebooks', 'df_test_pickle', 'df_train_pickle', 'df_val_pickle', 'image', 'text', 'text_embedding.py', 'text_embed_1.pt', 'text_embed.pt', 'meme.ipynb']


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.8 MB/s eta 0:00:00


In [6]:
# import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch

# import directories
import image
from text import text_embedding

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 2. Load dataset

In [7]:
# load labels_pd_pickle
train = pd.read_pickle('df_train_pickle')
val = pd.read_pickle('df_val_pickle')
test = pd.read_pickle('df_test_pickle')
# show first 5 rows
train['sarcastic']

0          not_sarcastic
1          not_sarcastic
2          not_sarcastic
3          not_sarcastic
4       little_sarcastic
              ...       
6995    little_sarcastic
6996      very_sarcastic
6997    little_sarcastic
6998       not_sarcastic
6999      very_sarcastic
Name: sarcastic, Length: 7000, dtype: object

In [21]:
train_data = text_embedding.dataloader(train)
val_data = text_embedding.dataloader(val)

In [22]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_data, batch_size=128, collate_fn=basic_collate_fn_all, shuffle=True)
dev_loader = DataLoader(val_data, batch_size=128, collate_fn=basic_collate_fn_all, shuffle=True)

## 3. Fine-tune RoBERTa

## 4. Train Model

In [17]:
# transformer for multi-tasking

import torch
import torch.nn as nn
import torch.optim as optim

class TransformerClassifier(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers):
        super(TransformerClassifier, self).__init__()
        self.linear1 = nn.Linear(768,512)
        self.linear2 = nn.Linear(200,512)
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(512, activation='gelu',nhead=8),6)
        self.encoder2 = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(200, activation='relu',dropout=0.2,nhead=8),3)
        self.fc = nn.Linear(512, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x,y):
        x = self.linear1(x)
        y = self.linear2(y)
        res = self.encoder(torch.stack((x,y)))
        res=  res.max(dim=0)[0]
        res = self.fc(res)
        return res


In [23]:
def train_tf(model, train_loader, optimizer, epochs=5):
    train_loss, train_loss_ind, val_loss, val_loss_ind = [], [], [], []
    best_acc = 0
    for epoch in range(epochs):
        model.train()
        for idx, (text, image, target) in enumerate(train_loader):
            optimizer.zero_grad()
            target = torch.Tensor(target).to('cuda')
            output = model(text,image)
            loss = nn.MSELoss()(output,target)
            loss.backward()
            optimizer.step()
        print('Epoch: {} Loss: {}'.format(
                    epoch, 
                    loss.item()))
        train_loss_ind.append(epoch)
        val_loss_ind.append(epoch)
        train_loss.append(loss.item())
        model.eval()
        acc = torch.Tensor([0,0,0,0,0]).to('cuda')
        with torch.no_grad():
          for idx, (text, image, target) in enumerate(dev_loader):
              target = torch.Tensor(target).to('cuda')
              output = model(text,image)
              loss = nn.MSELoss()(output,target)
              acc += sum(torch.round(output) == target) /1500
        print('Epoch',epoch,'val loss',loss.item(),'val acc',acc)
        val_loss.append(loss.item())
    stats = {'train_loss': train_loss,
          'train_loss_ind': train_loss_ind,
          'val_loss': val_loss,
          'val_loss_ind': val_loss_ind,
          'accuracy': acc,
    }
    return model,stats

In [ ]:
import matplotlib.pyplot as plt
from torch import optim

In [19]:
def plot_loss(stats):
    """Plot training loss and validation loss."""
    plt.plot(stats['train_loss_ind'], stats['train_loss'], label='Training loss')
    plt.plot(stats['val_loss_ind'], stats['val_loss'], label='Validation loss')
    plt.legend()
    plt.xlabel('Number of iterations')
    plt.ylabel('Loss')
    plt.show()

In [24]:
model_tf = TransformerClassifier(200,5,12).to('cuda')

In [25]:
optimizer = optim.Adam(model_tf.parameters(), lr=0.0001,weight_decay=0.001)
model, stats = train_tf(model_tf,train_loader,optimizer,50)
plot_loss(stats)

Epoch: 0 Loss: 0.3559109568595886
Epoch 0 val loss 0.46965211629867554 val acc tensor([0.4967, 0.2587, 0.7400, 0.9533, 0.8667], device='cuda:0')
Epoch: 1 Loss: 0.5157285928726196
Epoch 1 val loss 0.4239557385444641 val acc tensor([0.4967, 0.2587, 0.7400, 0.9533, 0.8667], device='cuda:0')
Epoch: 2 Loss: 0.4522130787372589
Epoch 2 val loss 0.47400954365730286 val acc tensor([0.4967, 0.2587, 0.7400, 0.9533, 0.8667], device='cuda:0')
Epoch: 3 Loss: 0.49131080508232117
Epoch 3 val loss 0.46907684206962585 val acc tensor([0.4967, 0.2587, 0.7393, 0.9533, 0.8667], device='cuda:0')
Epoch: 4 Loss: 0.5066961646080017
Epoch 4 val loss 0.5502907633781433 val acc tensor([0.4967, 0.2733, 0.4420, 0.9533, 0.8667], device='cuda:0')
Epoch: 5 Loss: 0.5511493682861328
Epoch 5 val loss 0.42201825976371765 val acc tensor([0.4967, 0.2587, 0.6527, 0.9533, 0.8667], device='cuda:0')
Epoch: 6 Loss: 0.4741969108581543
Epoch 6 val loss 0.4939471483230591 val acc tensor([0.2867, 0.2587, 0.7340, 0.9533, 0.8667], devi

NameError: ignored

In [18]:
def basic_collate_fn_all(batch):
    """Collate function for basic setting."""
    # texts
    text = torch.stack(tuple(item['text'] for item in batch)).to('cuda')
    # image
    image = torch.stack(tuple(item['image_embedding'] for item in batch)).to('cuda')
    # labels
    labels = [item['rating'] for item in batch]
    return text, image, labels


In [38]:
image_embedding = torch.load('image/VGG_basic_test.pt')

In [40]:
image_embedding.squeeze().shape

torch.Size([1500, 200])

In [ ]:
model = text_embedding.Finetuner()
model.load_state_dict(torch.load('text/roberta_ft1.pt'))
model.eval()

In [ ]:
model = torch.load('text/roberta_ft1.pt')

In [41]:
def get_test(model,test):
  image_embedding = torch.load('image/VGG_basic_test.pt').squeeze().to('cuda')
  text_embedding = torch.load('text_embed.pt')
  text = torch.stack(tuple(item for item in text_embedding)).to('cuda')
  result = model.forward(text,image_embedding)
  torch.save(torch.round(result),'result.pt')
get_test(model,test)

In [ ]:
a = tuple(i for i in range(10))

## 5. Test Model

## 6. Evaluate Model

In [ ]:
def eval(predictions):
    '''REMEMBER: predictions is a list, which contains predictions of humour, sarcasm, offensive, moltivational and overall_sentiment'''
    print("humour:",evaluation.evaluation(predictions[0],test['humour']))
    print("sarcasm:",evaluation.evaluation(predictions[1],test['sarcasm']))
    print("offensive:",evaluation.evaluation(predictions[2],test['offensive']))
    print("moltivational:",evaluation.evaluation(predictions[3],test['moltivational']))
    print("overall_sentiment",evaluation.evaluation(predictions[4],test['overall_sentiment']))